In [5]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [61]:
import pandas as  pd
import requests
import numpy as np
from flipside import Flipside
import os
from dotenv import load_dotenv


In [12]:
# Config
columns = ['unique_trader', 'platform', 'blockchain','trading_vol', 'vol_all_platform', 'interactions_count', 'last_interact_date', 'last_date_all_platform', 'activity']

## Perpetual Trades

In [50]:
load_dotenv()

query_id = {
    "vertex": 3319571,
    "hmx": 3322162,
    "mux":3331840,
    "lvl":3329599,
    "synthetix":3331775,
    "gmx":3326970,
    "dydx":3350149,
    "aevo":3332234,
    "gns":3329779,
    'hyperliquid':3351533
}
dune_api = os.getenv('DUNE_API')
df_all = pd.DataFrame()
for platform,id in query_id.items():
    url = f"https://api.dune.com/api/v1/query/{str(id)}/results?api_key={dune_api}"
    response = requests.get(url)
    df = pd.DataFrame(response.json())
    data = df.iloc[-1,-1]
    df= pd.DataFrame(data)
    df= df[columns]
    # download
    df.to_excel(f"dataset/traders_{platform}.xlsx", index= False)
    print(f"donwload of {platform} data")
    # concatenate
    df_all = pd.concat([df_all,df], axis=0)
    

df_all.to_excel("dataset/concatenate.xlsx", index = False)
df_all

donwload of vertex data
donwload of hmx data
donwload of mux data
donwload of lvl data
donwload of synthetix data
donwload of gmx data
donwload of dydx data
donwload of aevo data
donwload of gns data
donwload of hyperliquid data


,unique_trader,platform,blockchain,trading_vol,vol_all_platform,interactions_count,last_interact_date,last_date_all_platform,activity
0,0xaf5e2af64e5926875fa5f3d5e6ac272fc602224e,Vertex,arbitrum,1.135845e+10,None,112298,2024-01-06,None,active
1,0x5494c260fd71799414d174aa802750efa2dca5dd,Vertex,arbitrum,9.769726e+09,None,105567,2024-01-06,None,active
2,0x351d1d0ed37f6854bfc36ba24caa529688990f6e,Vertex,arbitrum,2.724428e+09,None,110840,2024-01-06,None,active
3,0xd604cfa918a7fcf24fd5789483323cdcb37b7fcf,Vertex,arbitrum,4.762204e+08,None,24047,2024-01-06,None,active
4,0x519e19eac9ead404343c68224dd2a22c5c25f992,Vertex,arbitrum,4.017094e+08,None,4454,2024-01-03,None,active
...,...,...,...,...,...,...,...,...,...
1528,0xcfe122de90515685f806e79b29ff9351ce764098,Hyperliquid,optimism,1.000016e+04,None,1,2023-12-05,None,active
1529,0x9866f56b785d38e67eefed374821d1d7ae9ef18a,Hyperliquid,optimism,1.000013e+04,None,1,2023-10-13,None,active
1530,0x0681e5bee2f420bc5a1754f09e0dedc0c5b4be85,Hyperliquid,optimism,1.000006e+04,None,2,2023-12-05,None,active
1531,0xc18120b4b22c78ea55f8a7acbd7d7082d3c73a2b,Hyperliquid,optimism,1.000002e+04,None,2,2023-12-08,None,active


In [51]:
# Final Layout

pivot_df = df_all.pivot(index='unique_trader', columns='platform', values='trading_vol').fillna(0)
pivot_df['Cumulative Trading Volume'] = df_all.groupby('unique_trader')['trading_vol'].sum()
pivot_df['Count of Platforms Used'] = df_all.groupby('unique_trader')['platform'].nunique()
pivot_df['Number of Interactions'] = df_all.groupby('unique_trader')['interactions_count'].sum()
pivot_df['Date of Last Interaction'] = df_all.groupby('unique_trader')['last_interact_date'].max()
pivot_df.reset_index(inplace=True)

# Renaming the trading volume cols
platform_columns = [col for col in pivot_df.columns if col not in ['unique_trader', 'Cumulative Trading Volume', 'Count of Platforms Used', 'Number of Interactions', 'Date of Last Interaction']]
for col in platform_columns:
    pivot_df.rename(columns={col: f'Trading Volume {col}'}, inplace=True)

pivot_df.to_excel("dataset/perpetual_dex.xlsx", index= False)
pivot_df


platform,unique_trader,Trading Volume AAEVO,Trading Volume DYDX,Trading Volume GMX Trade,Trading Volume GNS,Trading Volume HMX,Trading Volume Hyperliquid,Trading Volume LVL Finance,Trading Volume MUX,Trading Volume Synthetix,Trading Volume Vertex,Cumulative Trading Volume,Count of Platforms Used,Number of Interactions,Date of Last Interaction
0,0x00000000000057667bfefe3158822e6fb4058a6d,0.0,0.0,5.808477e+04,0.00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,5.808477e+04,1,3,2023-09-29
1,0x00000000000e8716b4e2d12400a01474cdfad760,0.0,0.0,0.000000e+00,0.00,0.0,0.0,7.527903e+06,0.0,0.0,0.000000e+00,7.527903e+06,1,1900,2023-09-27
2,0x000000000036277944858dfe6b9ae18c929e570c,0.0,0.0,0.000000e+00,126056.04,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,1.260560e+05,1,2,2023-12-31
3,0x00000000007cee88e01241c9db0a02b859db5b52,0.0,0.0,3.415701e+05,0.00,0.0,0.0,4.952040e+05,0.0,0.0,0.000000e+00,8.367742e+05,2,1648,2023-12-25
4,0x00000000051cbce3fd04148cce2c0adc7c651829,0.0,0.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0.0,2.548706e+04,2.548706e+04,1,6,2023-12-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44081,0xfff81c5b5882769f2a4b8a6537b7251c9bc00005,0.0,0.0,2.315707e+06,0.00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.315707e+06,1,36,2023-08-31
44082,0xfffb0c1155a3268e6f2976a87083037adf0455e4,0.0,0.0,2.111555e+04,0.00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.111555e+04,1,5,2023-08-16
44083,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,0.0,0.0,2.996989e+04,0.00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.996989e+04,1,15,2023-10-02
44084,0xfffb92fb89ae51c421018e42a5ca6e7336314438,0.0,0.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0.0,1.024285e+06,1.024285e+06,1,68,2023-11-16


## NFT Community

In [53]:
#Extract data prepared from Flipside
# Getting the cohort_analysis query
url = "https://flipsidecrypto.xyz/api/v1/queries/e3cced65-01d9-48b9-b164-71a2675e3990/data/latest"
response = requests.get(url)
data = response.json()
df_nft= pd.DataFrame(data)
df_nft

,WALLET,NFT_NAME,COUNT,WALLET_BALANCE_USD
0,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,CRYPTOPUNKS,829,1.19
1,0x29469395eaf6f95920e59f858042f0e28d98a20b,PudgyPenguins,440,10651.27
2,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,CRYPTOPUNKS,420,2458.25
3,0x29469395eaf6f95920e59f858042f0e28d98a20b,Kanpai Pandas,250,10651.27
4,0xcb2a8334143176e6eb4863e7ab3ed5f3258ee94b,Merit Circle - Tactile NFT,247,211834.42
...,...,...,...,...
14443,0x8b9f8a8a7c383f2d861cf05f35c28359cfbc97c3,lobsterdao,1,631.76
14444,0xff0613779201d09ec7f31f2fff5238d3998244e7,lobsterdao,1,774.98
14445,0xc4f370c6d3164a56971692362a9e488c0992a29d,CRYPTOPUNKS,1,1819.81
14446,0xec4f5fce4312936bae1da788d8ea595e13224df5,PudgyPenguins,1,3006.28


In [54]:
len(set(df_nft.WALLET.values))

13682

###  TOP LProviders  on Uniswap & Curve AN (Ethereem and Arbitrum)

In [48]:
url = "https://flipsidecrypto.xyz/api/v1/queries/28a95d3b-2c12-42e7-97bf-aa2ac4c30e41/data/latest"
response = requests.get(url)
data = response.json()
df_dex= pd.DataFrame(data)
df_dex

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT,IS_ARB_CONTRACT
0,0x0004d2a2f9a823c1a585fde6514a17ff695e0001,Uniswap & Curve,active,6.459790e+06,No,No
1,0x0000ba9ff5c97f33bd62c216a56b3d02ae6ac4bb,Uniswap & Curve,active,2.078376e+08,No,No
2,0x0000b8e312942521fb3bf278d2ef2458b0d3f243,Uniswap & Curve,active,2.265685e+09,No,No
3,0x000000002fcecf3bc87acc58802cdb526c4f019d,Uniswap & Curve,active,2.317281e+06,No,No
4,0x00000631f24bf8ae563c9ee18a053e408c9ef1af,Uniswap & Curve,active,2.407692e+06,No,No
5,0x003fd5f6030b8c1a4d2503339b14466b9ba6c6cf,Uniswap & Curve,active,3.173167e+09,No,No
6,0x0000a441fbb1fbaadf246539bf253a42abd31494,Uniswap & Curve,active,2.109899e+07,No,No
7,0x003db19049811190f330b20406fb1354fab27355,Uniswap & Curve,active,2.330483e+08,No,No
8,0x000013de30d1b1d830dcb7d54660f4778d2d4af5,Uniswap & Curve,active,4.462015e+09,No,No
9,0x00000000ff0027b3f3d2b0412d4a4ced4e3366da,Uniswap & Curve,active,5.776541e+06,Yes,No


### Staking Protocol such as Lido & Rocket Pool

In [49]:
url = "https://flipsidecrypto.xyz/api/v1/queries/b3239ae5-92c8-495d-99cc-9ac17b155dc8/data/latest"
response = requests.get(url)
data = response.json()
df_stake= pd.DataFrame(data)
df_stake

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0xdf6750ecdb74c0cc548d6536f4ec5d67e0376ed3,Lido & Rocketpool,active,2.116714e+06,Yes
1,0x1a4d6827cf83d3db1f43cd2e833842db71027225,Lido & Rocketpool,active,2.560770e+06,Yes
2,0x23ebcd701fd92867235aeb0174b7c444b9b2b3ad,Lido & Rocketpool,active,1.633834e+06,Yes
3,0xeaa7723633cf598e872d611f5ec50a45b65cbc72,Lido & Rocketpool,active,2.095581e+08,Yes
4,0x714508fb0ade6d82f94b21f261a9d9a96779ef0a,Lido & Rocketpool,active,2.858031e+07,Yes
...,...,...,...,...,...
1926,0xa6724083a78c106c9e6141217de40344744cf9d7,Lido & Rocketpool,active,1.583870e+06,No
1927,0x95b7c8384d49d12c7e8ad4d4128e258812f57fab,Lido & Rocketpool,active,3.539471e+06,No
1928,0xf8153167313ce9cfcb45bd4aff2b543513388163,Lido & Rocketpool,active,1.821165e+06,No
1929,0x3ad1b118813e71a6b2683fcb2044122fe195ac36,Lido & Rocketpool,active,1.502562e+07,No


### MAkerDAO gov

In [55]:
url = "https://flipsidecrypto.xyz/api/v1/queries/8a8bd36e-e0a5-4170-a3c5-7fd16a50dbe6/data/latest"
response = requests.get(url)
data = response.json()
df_maker= pd.DataFrame(data)
df_maker

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x8d4df847db7ffe0b46af084fe031f7691c6478c2,MakerDAO Gov.,active,None,No
1,0x4bb12cc382e36b4b6faf7bdca7708969aed258ef,MakerDAO Gov.,active,None,No
2,0x480ac654b35f9102b3a14ceb72ba375948c54acf,MakerDAO Gov.,active,None,No
3,0x45ea58092ba3bb828c3ac763b1701a432bd16fca,MakerDAO Gov.,active,None,Yes
4,0x4971dd016127f390a3ef6b956ff944d0e2e1e462,MakerDAO Gov.,active,None,Yes
...,...,...,...,...,...
311,0x016e9153a386d53c6fc0ca2d6a56dbb3834dc4b7,MakerDAO Gov.,active,None,No
312,0x1ccb144b700ec726d37db38c617e154de6d9c0d0,MakerDAO Gov.,active,None,No
313,0x7702629e39684daec232c5d830a34c45c98833dc,MakerDAO Gov.,active,None,No
314,0x24f356a9c4ddf1f003d786b7c2740c10583be53f,MakerDAO Gov.,active,None,Yes


### veCRV Holders

In [57]:
url = "https://flipsidecrypto.xyz/api/v1/queries/ba79099c-7652-4730-8e73-c077fd3e28bf/data/latest"
response = requests.get(url)
data = response.json()
df_crv= pd.DataFrame(data)
df_crv

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0xb2398330970c7cad95535d076ba3f64cc6afd43d,Curve Gov.,active,36777.161128,No
1,0x432dcbda06e8b296ca29705572d7cb6315ed8bed,Curve Gov.,active,10319.943538,No
2,0x7dc32a639ebb605d36f2c1076d5d74d69c254596,Curve Gov.,active,17425.734352,No
3,0x0991c61d1c55f2c77bb40be4ce223050fd0128ca,Curve Gov.,active,2451.250293,No
4,0x4f6fff5cd87eaf8cf09467366fe783e7ecb1317b,Curve Gov.,active,1466.316385,No
...,...,...,...,...,...
2777,0xc69750cc8e5f114d14aa8e5f92de994c612c58cd,Curve Gov.,active,2454.081402,No
2778,0xdbf5a8986e854fe07ffe4506e1d24d50fdf59974,Curve Gov.,active,4910.078619,No
2779,0x19ae3cf684087e2cb9cc2dd2b58c29f79f4e9d02,Curve Gov.,active,383523.884585,No
2780,0xfac2f11ba2577d5122dc1ec5301d35b16688251e,Curve Gov.,active,33941.914779,No


### Optimism Governance

In [ ]:
dune_api = os.getenv("DUNE_API")
url = f"https://api.dune.com/api/v1/query/3349830/results?api_key={dune_api}"
response = requests.get(url)
df = pd.DataFrame(response.json())
data = df.iloc[-1,-1]
df= pd.DataFrame(data)


In [72]:
df = df[['wallet', 'platform', 'activity', 'amount_usd', 'is_eth_contract']]
df.columns = df.columns.str.upper()
df_opt = df.copy()
df_opt

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x33107b56647014f37375b8291a86409f81a148db,Optimism,active,None,None
1,0x99b17456ed9c3eb3e2f4f292bcb515f8bb9f4640,Optimism,active,None,None
2,0xa01058ec7ce868a3dfaaeb1e2ce472c67a911478,Optimism,active,None,None
3,0xded935a602eac707e32b4155a055e2f6ac230bf2,Optimism,active,None,None
4,0x4af178f38f02f57144e23949d502cb510e721bd4,Optimism,active,None,None
...,...,...,...,...,...
20781,0x0e24d281364bbf1ee6d98d93d744ded873117efe,Optimism,active,None,None
20782,0x82c23fe118818870917a465759822106e4e44f82,Optimism,active,None,None
20783,0xe69a8de3bdd5218fbf6c14e8dc6bbbc6624434d0,Optimism,active,None,None
20784,0xb15f59dc6daf5c2a56bb6a7bf2e8e66795a0bec9,Optimism,active,None,None


### Governance (DEX/Stake/Chains)


In [88]:
url = "https://flipsidecrypto.xyz/api/v1/queries/631b19a0-46ac-4f2a-8a36-37be4cc41df2/data/latest"
response = requests.get(url)
data = response.json()
df_gov= pd.DataFrame(data)
df_gov['IS_ETH_CONTRACT'] = np.nan
df_gov

,WALLET,PLATFORM,ACTIVITY,AMOUN_USD,IS_ETH_CONTRACT
0,0x931f0fcbc61b46176ab5b8d27c3de8fbad27a99c,GMX Gov.,active,None,NaN
1,0xb2c4b7761babda70adb23604a71f2fb511afc9bf,GMX Gov.,active,None,NaN
2,0x8baaf753aa7fd611af42fe5dba8b3809e9b1fa7f,GMX Gov.,active,None,NaN
3,0x90d4cf762fd26ed339a6ce087de7a79f0d123e9f,GMX Gov.,active,None,NaN
4,0xbf041fe55449df61f9ace1cd656414e29aeed411,snxgov.eth,active,None,NaN
...,...,...,...,...,...
59449,0x24b304cb2c351415a31a2bdcedd689aeff2c3295,ENS Gov,active,None,NaN
59450,0x1217f7a669f50cf8db3d496e781235a102778b05,Gitcoin Gov,active,None,NaN
59451,0xee276d94423f8de7394de28b57c5e7fbb40b33e9,Starknet Gov.,active,None,NaN
59452,0xfb7397832cadaf2685c7704012648e7cde23ce60,Gitcoin Gov,active,None,NaN


### Gitcoin Grant Donors

In [89]:
dune_api = os.getenv("DUNE_API")
url = f"https://api.dune.com/api/v1/query/3351720/results?api_key={dune_api}"
response = requests.get(url)
df = pd.DataFrame(response.json())
data = df.iloc[-1,-1]
df_gitcoin= pd.DataFrame(data)
df_gitcoin = df_gitcoin[['wallet', 'platform', 'activity', 'amount_usd', 'is_eth_contract']]
df_gitcoin.columns = df_gitcoin.columns.str.upper()
df_gitcoin

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x8bab28f68b87d10473299a9bb713820ae7b63dde,Gitcoin Grants,active,None,None
1,0xf52ce53468ae82581f85387863e4d86cadc9265a,Gitcoin Grants,active,None,None
2,0xa48aea91cc19e211e9754414416ac9cdc702e210,Gitcoin Grants,active,None,None
3,0x4e61a2ec8b12a7557b78698ea75bde6c02eb6668,Gitcoin Grants,active,None,None
4,0xa7a1a69e9562df928c35ab2954ffd3c9d42d5f57,Gitcoin Grants,active,None,None
...,...,...,...,...,...
528,0x75f7e365afa2830a1e108dda505a539511f9cb5f,Gitcoin Grants,active,None,None
529,0x659d3c1e42c3ab437ff542699e207ecdac3e1afa,Gitcoin Grants,active,None,None
530,0xf7b028006a3acb8a2293f4b895a9a6fa0bf071eb,Gitcoin Grants,active,None,None
531,0x75d5fe7b9a6f56191a60480a5816575875f6fce9,Gitcoin Grants,active,None,None


## Final Layouts

In [92]:
df_og = pd.concat([df_stake, df_dex, df_maker, df_crv, df_opt, df_gov, df_gitcoin], axis = 0)
df_og

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT,IS_ARB_CONTRACT,AMOUN_USD
0,0xdf6750ecdb74c0cc548d6536f4ec5d67e0376ed3,Lido & Rocketpool,active,2.116714e+06,Yes,NaN,NaN
1,0x1a4d6827cf83d3db1f43cd2e833842db71027225,Lido & Rocketpool,active,2.560770e+06,Yes,NaN,NaN
2,0x23ebcd701fd92867235aeb0174b7c444b9b2b3ad,Lido & Rocketpool,active,1.633834e+06,Yes,NaN,NaN
3,0xeaa7723633cf598e872d611f5ec50a45b65cbc72,Lido & Rocketpool,active,2.095581e+08,Yes,NaN,NaN
4,0x714508fb0ade6d82f94b21f261a9d9a96779ef0a,Lido & Rocketpool,active,2.858031e+07,Yes,NaN,NaN
...,...,...,...,...,...,...,...
528,0x75f7e365afa2830a1e108dda505a539511f9cb5f,Gitcoin Grants,active,NaN,None,NaN,NaN
529,0x659d3c1e42c3ab437ff542699e207ecdac3e1afa,Gitcoin Grants,active,NaN,None,NaN,NaN
530,0xf7b028006a3acb8a2293f4b895a9a6fa0bf071eb,Gitcoin Grants,active,NaN,None,NaN,NaN
531,0x75d5fe7b9a6f56191a60480a5816575875f6fce9,Gitcoin Grants,active,NaN,None,NaN,NaN


In [91]:
# NFT community final layout
perp_traders =  list(set(pivot_df.unique_trader.values))
og_whale_list =  list(set(df_og.WALLET.values))

df_nft['is_perp_trader'] = df_nft.WALLET.apply(lambda x: 'Yes' if x in perp_traders else 'No')
df_nft['is_og_whale'] = df_nft.WALLET.apply(lambda x: 'Yes' if x in og_whale_list else 'No')
df_nft.to_excel('dataset/nft_community.xlsx', index = False)
df_nft

,WALLET,NFT_NAME,COUNT,WALLET_BALANCE_USD,is_perp_trader,is_og_whale
0,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,CRYPTOPUNKS,829,1.19,No,No
1,0x29469395eaf6f95920e59f858042f0e28d98a20b,PudgyPenguins,440,10651.27,No,No
2,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,CRYPTOPUNKS,420,2458.25,No,No
3,0x29469395eaf6f95920e59f858042f0e28d98a20b,Kanpai Pandas,250,10651.27,No,No
4,0xcb2a8334143176e6eb4863e7ab3ed5f3258ee94b,Merit Circle - Tactile NFT,247,211834.42,No,No
...,...,...,...,...,...,...
14443,0x8b9f8a8a7c383f2d861cf05f35c28359cfbc97c3,lobsterdao,1,631.76,No,No
14444,0xff0613779201d09ec7f31f2fff5238d3998244e7,lobsterdao,1,774.98,No,No
14445,0xc4f370c6d3164a56971692362a9e488c0992a29d,CRYPTOPUNKS,1,1819.81,No,No
14446,0xec4f5fce4312936bae1da788d8ea595e13224df5,PudgyPenguins,1,3006.28,No,No
